In [ ]:
import genemunge
import json
import matplotlib
%matplotlib inline

# Searching for genes 

In [ ]:
# set up an object to search the gene ontology
searcher = genemunge.search.Searcher()

# get all of the GO categories associated with the word 'immune'
# set exact = False to walk through the ontology and grab all child terms
immune_categories = searcher.keyword_search(['immune'], exact=False)

# get all of the genes assigned to the immune_categories
immune_genes = searcher.get_genes(immune_categories)

# get a list of housekeeping genes
housekeeping = searcher.get_housekeeping_genes()

# keep all of the immune related genes that are not housekeeping genes
variable_immune_genes = list(set(immune_genes) - set(housekeeping))

print('Identified {} variable immune related genes'.format(len(variable_immune_genes)))

# Converting between identifier types

In [ ]:
# set up an object to convert from ensembl to symbol
ensembl_to_symbol = genemunge.convert.IDConverter('ensembl_gene_id', 'symbol')

# convert the immune identifiers to gene symbols
variable_immune_symbols = ensembl_to_symbol.convert_list(variable_immune_genes)

# Getting information about a specific gene

In [ ]:
# set up an object to describe genes
describer = genemunge.describe.Describer('symbol')

# get some basic information about one of the immune genes
print(json.dumps(describer.get_gene_info(variable_immune_symbols[0]), indent=2))

# make a plot of the expression of one of the immune genes across tissues from GTEx
describer.plot_tissue_expression(variable_immune_symbols[0], sortby='median')